In [62]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import matplotlib.pyplot as plt
import re
import cv2
import os
import urllib
import numpy as np
import pandas as pd
import sqlite3
from math import sin, cos, sqrt, atan2, radians

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: 
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp
    
    
def mydistance(a1,b1,a2,b2):
    x1 = radians(a1)
    y1 = radians(b1)
    x2 = radians(a2)
    y2 = radians(b2)
    dlon = x2 - x1
    dlat = y2 - y1
    a = sin(dlat / 2)**2 + cos(y1) * cos(y2) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))    
    return c


def letterbox(img, new_shape=(240, 320), color=(114, 114, 114)):
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])

    ratio = r, r  
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  
    # minimum rectangle
    dw, dh = np.mod(dw, 64), np.mod(dh, 64)  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

def labelvs(k):
    if k <200:
        a = 'Dense'
    elif k < 1000:
        a = 'Shallow'
    else:
        a = 'None'
    return a

def weatherlabel(rain, fog):
    if rain != 0:
        return '비'
    elif fog == 'Dense' or fog == 'Shallow' :
        return '안개'
    else:
        return '맑음'
    

def sunnylabel(k):
    if k == '맑음':
        return 1
    return 0

In [63]:
import datetime

In [64]:
today = datetime.datetime.today()
today1 = today.strftime('%Y%m%d%H')

In [18]:
from datetime import date, timedelta

yesterday = date.today() - timedelta(1)
yesterday = yesterday.strftime('%Y%m%d')
yesterday

'20201104'

In [65]:
url = 'http://openapi.its.go.kr:8081/api/NCCTVInfo'
api_key = '1602398376506'
params= {
    'key':api_key,
    'ReqType':'2',
    'type ':'its',
    'MinX' : 126.449010,
    'MaxX' : 127.340151,
    'MinY' : 37.008739,
    'MaxY' : 37.773444,
}
resp=download(url,params, headers,'POST')

In [66]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [67]:
col = ['roadsectionid','filecreatetime','cctvtype','cctvurl','cctvresolution','coordy','cctvformat','cctvname','coordx']
row = []
rows = []
for i in dom.find_all('data'):
    for co in col:
        k = i.find(co).text.strip()
        row.append(k)
    rows.append(row)
    row = []

In [68]:
import pandas as pd
data = pd.DataFrame(rows,columns=col)
data.head()

,roadsectionid,filecreatetime,cctvtype,cctvurl,cctvresolution,coordy,cctvformat,cctvname,coordx
0,,,2,http://cctvsec.ktict.co.kr/1/biQVXUi6lrO0Cl/2B...,,37.40665,MP4,[수도권제1순환선] 판교분기점2,127.09706
1,,,2,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,,37.42889,MP4,[수도권제1순환선] 성남,127.12361
2,,,2,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,,37.43909,MP4,[수도권제1순환선] 성남영업소,127.12262
3,,,2,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,,37.475,MP4,[수도권제1순환선] 송파,127.12944
4,,,2,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,,37.51167,MP4,[수도권제1순환선] 서하남,127.14972


In [69]:
data['coordy'] = data['coordy'].astype('float32')
data['coordx'] = data['coordx'].astype('float32')
data['cctvname'] = data['cctvname'].str.replace('/','_')
sum(data['cctvname'].str.contains('\/'))

0

In [70]:
for i in col:
    print(sum(data[i]==''))
    break

587


In [71]:
data = data.drop(['roadsectionid','filecreatetime','cctvresolution','cctvformat','cctvtype'],axis=1)
data.head()

,cctvurl,coordy,cctvname,coordx
0,http://cctvsec.ktict.co.kr/1/biQVXUi6lrO0Cl/2B...,37.406651,[수도권제1순환선] 판교분기점2,127.097061
1,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,37.428890,[수도권제1순환선] 성남,127.123611
2,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,37.439091,[수도권제1순환선] 성남영업소,127.122620
3,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,37.474998,[수도권제1순환선] 송파,127.129440
4,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,37.511669,[수도권제1순환선] 서하남,127.149719


In [187]:
data[data['cctvname']=='[수도권제1순환선] 서하남']

,cctvurl,coordy,cctvname,coordx
4,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,37.511669,[수도권제1순환선] 서하남,127.149719
388,http://cctvsec.ktict.co.kr/3956/2MN1glES6E6ndf...,37.506447,[수도권제1순환선] 서하남,127.145569


In [215]:
data['cctvname'].value_counts()[:10]

[안양성남선] 청계산4터널(성남)    2
[안양성남선] 삼성산터널(성남)     2
[안양성남선] 청계산3터널(안양)    2
[제2외곽순환선] 북청라대교       2
[안양성남선] 삼성산터널(안양)     2
[안양성남선] 청계산3터널(성남)    2
[수도권제1순환선] 서하남        2
[경부선] 안성분기점           2
[영동선] 양상교             1
[서해안선] 양노4교           1
Name: cctvname, dtype: int64

In [189]:
data[['cctvname','coordy','coordx']].to_csv('CCTV.csv',index=False,encoding='CP949')

In [72]:
if not os.path.isdir(today1):
    os.makedirs(today1)
for url,name in zip(data['cctvurl'],data['cctvname']):
    urllib.request.urlretrieve(url,today1+'/'+ name + '.mp4')

HTTPError: HTTP Error 403: Forbidden

In [10]:
data = pd.read_csv('CCTV.csv',encoding='CP949')
data

,cctvname,coordy,coordx
0,[수도권제1순환선] 판교분기점2,37.406650,127.097060
1,[수도권제1순환선] 성남,37.428890,127.123610
2,[수도권제1순환선] 성남영업소,37.439090,127.122620
3,[수도권제1순환선] 송파,37.475000,127.129440
4,[수도권제1순환선] 서하남,37.511670,127.149720
...,...,...,...
560,[수도권제1순환선] 금토,37.394720,127.069440
561,[경인선] 효성,37.523643,126.704765
562,[경인선] 신월2,37.524418,126.828170
563,[영동선] 서창,37.417778,126.738050


In [73]:
if not os.path.isdir(today1):
    os.makedirs(today1)
for url,name in zip(data['cctvurl'],data['cctvname']):
    urllib.request.urlretrieve(url,today1+'/'+ name + '.mp4')

In [153]:
data.head()

,cctvurl,coordy,cctvname,coordx
0,http://cctvsec.ktict.co.kr/1/biQVXUi6lrO0Cl/2B...,37.406651,[수도권제1순환선] 판교분기점2,127.097061
1,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,37.428890,[수도권제1순환선] 성남,127.123611
2,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,37.439091,[수도권제1순환선] 성남영업소,127.122620
3,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,37.474998,[수도권제1순환선] 송파,127.129440
4,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,37.511669,[수도권제1순환선] 서하남,127.149719


'2020110505'

In [149]:
n = len(os.listdir(today1))
imgs = [None]*n
for i , _ in enumerate(os.listdir(today1)):
    img = [None]*10
    cap = cv2.VideoCapture(today1 + '/' + _)
    assert cap.isOpened(), 'Failed to open %s' % _
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    for j in range(10):
        ret, image = cap.read()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img[j] = gray
    s = np.stack([letterbox(x)[0] for x in img], 0) 
    imgs[i] = s

In [22]:
len(os.listdir('2020110217'))

557

In [192]:
for x in imgs:
    if x.shape != (10,240,320,3):
        print(x.shape)

In [225]:
url = 'https://www.weather.go.kr/cgi-bin/aws/nph-aws_txt_min_guide_test?'+ today1 +'00&MINDB_01M&0&m&K'
resp=download(url,  headers = headers,method= 'POST')
dom = BeautifulSoup(resp.content, 'lxml')
k =pd.read_html(str(dom.find('table')),header=0)
today_weather = k[1][['AWS 지점 / 고도','AWS 지점 / 고도.1','강수','강수15','위치']]
today_weather['강수15'].replace('.','0',inplace=True)
today_weather['강수15'] = today_weather['강수15'].astype('float')

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\pandas\core\series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [226]:
today_weather['강수15'].unique()

array([0.])

In [123]:
today_weather.to_csv('AWS_measure.csv',index=False,encoding='CP949')

In [24]:
con = sqlite3.connect("meta_aws.db")
cur = con.cursor()
query = cur.execute('select * from test')
cols = [column[0] for column in query.description]
ee = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
cur.close()
con.close()
ee[['지점','지점명','위도','경도']]

,지점,지점명,위도,경도
0,116,관악(레),37.4453,126.9640
1,326,용문산,37.5455,127.6113
2,400,강남,37.4982,127.0816
3,401,서초,37.4846,127.0260
4,402,강동,37.5555,127.1450
...,...,...,...,...
90,665,무의도,37.3904,126.4252
91,692,백학,38.0404,126.9245
92,889,현충원,37.5004,126.9765
93,966,풍도,37.1161,126.3856


In [227]:
AWS = pd.merge(today_weather,ee,left_on='AWS 지점 / 고도.1',right_on='지점명')[['AWS 지점 / 고도','AWS 지점 / 고도.1','강수','강수15','위치','위도','경도']]
AWS

,AWS 지점 / 고도,AWS 지점 / 고도.1,강수,강수15,위치,위도,경도
0,116,관악(레),○,0.0,경기도 과천시 중앙동,37.4453,126.9640
1,326,용문산,○,0.0,경기도 양평군 용문면 중원리,37.5455,127.6113
2,400,강남,○,0.0,서울특별시 강남구 일원동,37.4982,127.0816
3,401,서초,○,0.0,서울특별시 서초구 서초동,37.4846,127.0260
4,402,강동,○,0.0,서울특별시 강동구 고덕동,37.5555,127.1450
...,...,...,...,...,...,...,...
86,665,무의도,○,0.0,인천광역시 중구 무의동,37.3904,126.4252
87,692,백학,○,0.0,경기도 연천군 백학면 두일리,38.0404,126.9245
88,889,현충원,○,0.0,서울특별시 동작구 사당동,37.5004,126.9765
89,966,풍도,-,0.0,경기도 안산시단원구 대부남동,37.1161,126.3856


In [35]:
AWS[['AWS 지점 / 고도.1','강수','강수15','위도','경도']]

NameError: name 'AWS' is not defined

In [163]:
today1

'2020110217'

In [6]:
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp

In [7]:
servicekey = 'yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D'
# url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&ServiceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&numOfRows=100&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt='+yesterday+'&startHh=01&endDt='+yesterday+'&endHh=23&stnIds='

con = sqlite3.connect("meta_aws.db")

cur = con.cursor()
query = cur.execute('select * from meta_asos')
cols = [column[0] for column in query.description]
rr = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
cur.close()
con.close()

asos = pd.DataFrame()
for _ in rr['지점']:
    resp=download(url + str(_),headers = headers,method = 'GET')
    result= resp.json()
    asos_ = pd.DataFrame.from_dict(result['response']['body']['items']['item'])
    asos_ = asos_[['stnNm','tm','vs','stnId']]
    asos = pd.concat([asos,asos_])
asos.reset_index(drop=True, inplace=True)
asos['vs'] = asos['vs'].astype('int')

[Error] http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&ServiceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&numOfRows=100&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt=20201104&startHh=01&endDt=20201104&endHh=23&stnIds=98
[Error] http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&ServiceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&numOfRows=100&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt=20201104&startHh=01&endDt=20201104&endHh=23&stnIds=99
[Error] http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&Servi

In [8]:
asos['fog'] = asos['vs'].map(lambda x : labelvs(x))
ASOS = pd.merge(asos, rr, left_on='stnNm', right_on='지점명')[['지점','지점명','tm','fog','위도','경도']]
ASOS['tm'] = pd.to_datetime(ASOS['tm'])
ASOS['tm'] = ASOS['tm'].dt.strftime('%Y%m%d%H')
# ASOS = ASOS[ASOS['tm']==yesterday+'07']
# ASOS.reset_index(drop=True, inplace=True)

In [9]:
yes = ['2020110407','2020110417']
ASOS[ASOS['tm'].str.contains(yes[0]+'|'+yes[1])]

,지점,지점명,tm,fog,위도,경도
6,98,동두천,2020110407,None,37.9019,127.0607
15,99,파주,2020110407,None,37.8859,126.7665
24,102,백령도,2020110407,None,37.9740,124.7124
33,108,서울,2020110407,None,37.5714,126.9658
42,112,인천,2020110407,None,37.4777,126.6249
51,119,수원,2020110407,None,37.2575,126.9830
60,201,강화,2020110407,None,37.7074,126.4463
69,202,양평,2020110407,None,37.4886,127.4945
78,203,이천,2020110407,None,37.2640,127.4842


In [34]:
asos[asos['stnNm'].isin(['서울','인천','수원'])].iloc[[0,9,18],:]

,stnNm,tm,vs,stnId,fog
27,서울,2020-11-04 01:00,2000,108,None
36,인천,2020-11-04 01:00,2000,112,None
45,수원,2020-11-04 01:00,2000,119,None


In [26]:
rr

,index,지점,지점명,위도,경도
0,0,98,동두천,37.9019,127.0607
1,1,99,파주,37.8859,126.7665
2,2,102,백령도,37.9740,124.7124
3,3,108,서울,37.5714,126.9658
4,4,112,인천,37.4777,126.6249
5,5,119,수원,37.2575,126.9830
6,6,201,강화,37.7074,126.4463
7,7,202,양평,37.4886,127.4945
8,8,203,이천,37.2640,127.4842


In [249]:
data['AWS 지점'] = ''
data['ASOS 지점'] = ''
num = 0
for i,j in zip(data['coordx'],data['coordy']):
    distance = pd.Series(AWS[['위도','경도']].apply (lambda row: mydistance(i,j,row['경도'],row['위도']), axis=1))
    data.loc[num,'AWS 지점'] = AWS.loc[np.argmin(distance),'AWS 지점 / 고도.1']
    distance = pd.Series(ASOS[['위도','경도']].apply (lambda row: mydistance(i,j,row['경도'],row['위도']), axis=1))
    data.loc[num,'ASOS 지점'] = ASOS.loc[np.argmin(distance),'지점명']
    num+=1
    
data = pd.merge(data,AWS,left_on = 'AWS 지점',right_on = 'AWS 지점 / 고도.1')
data = pd.merge(data,ASOS,left_on = 'ASOS 지점',right_on = '지점명')
data['weather'] = data.apply(lambda x : weatherlabel(x['강수15'],x['fog']),axis=1)
data['sunny'] = data['weather'].map(lambda x : sunnylabel(x))
data.head()

,cctvurl,coordy,cctvname,coordx,AWS 지점,ASOS 지점,AWS 지점 / 고도,AWS 지점 / 고도.1,강수,강수15,...,위도_x,경도_x,지점,지점명,tm,fog,위도_y,경도_y,weather,sunny
0,http://cctvsec.ktict.co.kr/1/biQVXUi6lrO0Cl/2B...,37.406651,[수도권제1순환선] 판교분기점2,127.097061,성남,수원,572,성남,○,0.0,...,37.4209,127.1248,119,수원,2020110217,None,37.2575,126.983,맑음,1
1,http://cctvsec.ktict.co.kr/15/6nct8jLCs5mUWXmv...,37.394718,[수도권제1순환선] 청계운중교,127.069443,성남,수원,572,성남,○,0.0,...,37.4209,127.1248,119,수원,2020110217,None,37.2575,126.983,맑음,1
2,http://cctvsec.ktict.co.kr/77/VsbM0jxGQssbkGRf...,37.332779,[경부선] 죽전휴게소(서울),127.103607,성남,수원,572,성남,○,0.0,...,37.4209,127.1248,119,수원,2020110217,None,37.2575,126.983,맑음,1
3,http://cctvsec.ktict.co.kr/78/r2fNEgEff7Jvxpgr...,37.365559,[경부선] 서울영업소,127.102501,성남,수원,572,성남,○,0.0,...,37.4209,127.1248,119,수원,2020110217,None,37.2575,126.983,맑음,1
4,http://cctvsec.ktict.co.kr/79/sU0luP+CC0AXAUzR...,37.399693,[경부선] 판교2,127.100433,성남,수원,572,성남,○,0.0,...,37.4209,127.1248,119,수원,2020110217,None,37.2575,126.983,맑음,1


In [253]:
data['sunny'].unique()

array([1], dtype=int64)

In [89]:
ASOS = pd.merge(asos, rr, left_on='stnNm', right_on='지점명')[['지점','지점명','tm','fog','위도','경도']]

In [136]:
ASOS[ASOS['tm'].str.startswith('20201025')]

,지점,지점명,tm,fog,위도,경도
6,98,동두천,2020102507,None,37.9019,127.0607
29,99,파주,2020102507,None,37.8859,126.7665
52,102,백령도,2020102507,None,37.9740,124.7124
75,108,서울,2020102507,None,37.5714,126.9658
98,112,인천,2020102507,None,37.4777,126.6249
121,119,수원,2020102507,shallow,37.2575,126.9830
144,201,강화,2020102507,None,37.7074,126.4463
167,202,양평,2020102507,None,37.4886,127.4945
190,203,이천,2020102507,shallow,37.2640,127.4842


In [12]:
from pathlib import Path
from itertools import islice

space =  '    '
branch = '│   '
tee =    '├── '
last =   '└── '

def tree(dir_path: Path, level: int=-1, limit_to_directories: bool=False,
         length_limit: int=1000):
    """Given a directory Path object print a visual tree structure"""
    dir_path = Path(dir_path) # accept string coerceable to Path
    files = 0
    directories = 0
    def inner(dir_path: Path, prefix: str='', level=-1):
        nonlocal files, directories
        if not level: 
            return # 0, stop iterating
        if limit_to_directories:
            contents = [d for d in dir_path.iterdir() if d.is_dir()]
        else: 
            contents = list(dir_path.iterdir())
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + path.name
                directories += 1
                extension = branch if pointer == tee else space 
                yield from inner(path, prefix=prefix+extension, level=level-1)
            elif not limit_to_directories:
                yield prefix + pointer + path.name
                files += 1
    print(dir_path.name)
    iterator = inner(dir_path, level=level)
    for line in islice(iterator, length_limit):
        print(line)
    if next(iterator, None):
        print(f'... length_limit, {length_limit}, reached, counted:')
    print(f'\n{directories} directories' + (f', {files} files' if files else ''))

In [14]:
tree(os.getcwd() + '/cctv_aws', level=2,limit_to_directories=True)

cctv_aws
├── .idea
│   └── inspectionProfiles
├── csv
├── image
├── venv
│   ├── Include
│   ├── Lib
│   └── Scripts
└── video
    ├── 2020102517
    ├── 2020102612
    ├── 2020102617
    ├── 2020102707
    ├── 2020102717
    ├── 2020102917
    ├── 2020103108
    ├── 2020110114
    ├── 2020110117
    ├── 2020110207
    └── 2020110217

20 directories
